**Table of contents**<a id='toc0_'></a>    
- [Search Engine With Tools And Agents](#toc1_1_1_)    
  - [Arxiv - Wikipedia API Wrapper](#toc1_2_)    
  - [RAG Tool (tools=[wiki,arxiv,retriever_tool])    ](#toc1_3_)    
  - [Run all this tools with Agents and LLM Models](#toc1_4_)    
  - [LLM](#toc1_5_)    
  - [Prompt](#toc1_6_)    
  - [Agents (llm,tools,prompt)](#toc1_7_)    
- [Ek Notes:](#toc2_)    
  - [-> DuckDuckGoSearchRun](#toc2_1_)    
  - [-> StreamlitCallbackHandler](#toc2_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_1_1_'></a>[Search Engine With Tools And Agents](#toc0_)

- This notebook shows how to create a search engine that combines LangChain tools and agents with language models. It uses DuckDuckGo, Arxiv, and Wikipedia to search and provide relevant information to user queries.

- Environment Setup: Load environment variables and set up configurations for embedding models.
    * **Environment Configuration**: Uses `dotenv` to manage environment variables and includes commented code for HuggingFace embedding model setup.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# # for HF embedding model
# os.environ('HF_TOKEN') = os.getenv('HF_TOKEN')

# from langchain_huggingface import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-v2')


True

## <a id='toc1_2_'></a>[Arxiv - Wikipedia API Wrapper](#toc0_)

- Research Tools Setup: Initialize Arxiv and Wikipedia tools for accessing academic papers and encyclopedia entries.
    * **Tool Configuration**: Uses `ArxivQueryRun` and `WikipediaQueryRun` from LangChain community tools, along with their respective API wrappers for functionality.

In [2]:
## Arxiv--Research
## Tools creation

from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [3]:
# tool of wikipedia
# Initialize Wikipedia tool with parameters to retrieve limited, focused content per search.

api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)

wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

wiki.name

'wikipedia'

In [4]:
#tool of Arxiv

api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)

arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

print(arxiv.name)

arxiv


In [5]:
tools=[wiki,arxiv]

## <a id='toc1_3_'></a>[RAG Tool (tools=[wiki,arxiv,retriever_tool])](#toc0_)     [&#8593;](#toc0_)

In [6]:
## Custom tools[RAG Tool]

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


- Set up a document processing pipeline: load webpage data, split into chunks, create vector embeddings, and prepare for retrieval.

In [ ]:
# load data
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()

#split to chunks 
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

# create vectorstore(docs, embedModel)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings()) # default 'text-embedding-ada-002'

# retriever e cevir (chain olusturabilmek icin)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001EB64002570>, search_kwargs={})

- Create a retriever tool from the vector database for searching Langsmith documentation.

In [8]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

In [ ]:
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\duygu\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001EB57B99260>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object

## <a id='toc1_4_'></a>[Run all this tools with Agents and LLM Models](#toc0_)

## <a id='toc1_5_'></a>[LLM](#toc0_)

In [ ]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os

groq_api_key=os.getenv("GROQ_API_KEY")     # for llm
openai.api_key=os.getenv("OPENAI_API_KEY") # for embedding

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

## <a id='toc1_6_'></a>[Prompt](#toc0_)

- Load a pre-defined prompt template from LangChain Hub for OpenAI functions agent.

In [13]:
## Prompt Template

from langchain import hub

prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

https://smith.langchain.com/hub/hwchase17/openai-functions-agent

- This code pulls a specialized prompt template ("hwchase17/openai-functions-agent") from LangChain Hub, which is designed for OpenAI's function calling capabilities. The template includes pre-tested, community-developed prompt structures that optimize AI interactions and ensure consistent function calling behaviors in applications.


## <a id='toc1_7_'></a>[Agents (llm,tools,prompt)](#toc0_)

- Initialize an OpenAI tools agent by combining the language model, tools, and prompt template.

In [ ]:
## Create Agents

from langchain.agents import create_openai_tools_agent

agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

- Create an agent executor to manage and run the agent with specified tools and verbose output.

In [15]:
## Agent Executer

from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [16]:
agent_executor.invoke({"input":"Tell me about Langsmith"}) 



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Cloud)AdministrationSelf-hostingReferenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!
1. Install LangSmith‚Äã
PythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai
2. Create an API key‚Äã
To create an API key head to the Settings page. Then click Create API Key.
3. Set up your environment‚Äã
Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though

{'input': 'Tell me about Langsmith',
 'output': 'Based on the output provided by the tool, it appears that Langsmith is a platform for building production-grade LLM (Large Language Model) applications. It allows users to closely monitor and evaluate their applications, making it easier to ship quickly and with confidence.\n\nThe output provides a step-by-step guide on how to get started with Langsmith, including installing the platform, creating an API key, setting up the environment, and logging the first trace. It also provides information on how to run the first evaluation and access the Langsmith experiments UI.\n\nAdditionally, the output mentions that Langsmith is not necessary to use if the application is built on LangChain/LangGraph (either Python or JavaScript), and provides a link to a how-to guide for tracing with LangChain.\n\nOverall, Langsmith appears to be a powerful tool for building and deploying LLM applications, and the output provides a comprehensive guide on how to

In [17]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks witBased on the result of the tool call, I can continue to respond.

Machine learning (ML) is a subfield of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, without being explicitly programmed. The goal of machine learning is to develop models that can accurately predict or make decisions based on the patterns and relationships present in the data.

Machine learning is often used in applications such as image and speech recognition, natural language processing, and predictive analytics. One of the key advantages of machine learning is that it can learn from large amo

{'input': 'What is machine learning',
 'output': 'Based on the result of the tool call, I can continue to respond.\n\nMachine learning (ML) is a subfield of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, without being explicitly programmed. The goal of machine learning is to develop models that can accurately predict or make decisions based on the patterns and relationships present in the data.\n\nMachine learning is often used in applications such as image and speech recognition, natural language processing, and predictive analytics. One of the key advantages of machine learning is that it can learn from large amounts of data and improve its performance over time, without requiring explicit programming or human intervention.\n\nIn order to achieve this, machine learning algorithms use various techniques such as supervised learning, unsupervised learning, and reinforcement learning. Supervised learning invo

In [18]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,The paper "Attention Is All You Need" introduced the transformer architecture, which revolutionized the field of natural language processing.

> Finished chain.


{'input': "What's the paper 1706.03762 about?",
 'output': 'The paper "Attention Is All You Need" introduced the transformer architecture, which revolutionized the field of natural language processing.'}

In [ ]:
# END